In [159]:
%load_ext autoreload
%autoreload 2

from NewsContent import *
from UserContent import *
from preprocessing import *
from PEGenerator import *
import PEGenerator
from models import *
from utils import *
from Encoders import *

import os
import numpy as np
import json
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [160]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [161]:
data_root_path = "../data/Challenge/"
embedding_path = "../../"
KG_root_path = "../data/Challenge/entity"
popularity_path = "../data/Challenge/popularity"
config = {'title_length':30,
              'body_length':100,
              'max_clicked_news':50,
              'npratio':1,
              'news_encoder_name':"CNN",
              'user_encoder_name':"Att",
             'attrs':['title', 'entity','vert'],
             'word_filter':0,
             'data_root_path':data_root_path,
             'embedding_path':embedding_path,
             'KG_root_path':KG_root_path,
            'popularity_path':popularity_path,
            'batch_size': 32,
             'max_entity_num':5}
model_config = {
        'news_encoder':1,
        'popularity_user_modeling':True,
        'rel':True,
        'ctr':True,
        'content':True,
        'rece_emb':True,
        'activity':True

    }

In [162]:
News = NewsContent(config)

TrainUsers = UserContent(News.news_index,config,'train.tsv',2)
ValidUsers = UserContent(News.news_index,config,'val.tsv',1)
TestUsers = UserContent(News.news_index,config,'test.tsv',2)

19779
2472
2473


In [163]:
train_sess,train_buckets, train_user_id, train_label = get_train_input(TrainUsers.session,News.news_index,config)
test_impressions, test_userids = get_test_input(TestUsers.session,News.news_index)
val_impressions, val_userids = get_test_input(ValidUsers.session,News.news_index)

19779
19916
2473
2472


In [164]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,News.word_dict)
entity_embedding_matrix, have_word2 = load_matrix(embedding_path, News.entity_dict)

In [165]:
train_loader = DataLoader(TrainDataset(News, TrainUsers, train_sess, train_user_id, train_buckets, train_label), config['batch_size'])
val_user_data = UserDataset(News,ValidUsers)
test_user_data = UserDataset(News,TestUsers)
news_data = NewsDataset(News)

In [166]:
from torch.optim import Adam

model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = \
create_pe_model(config, model_config, News, title_word_embedding_matrix, entity_embedding_matrix, device)

model.to(device)
for name, module in model.named_children():
    module.to(device)
    print(f'Moved {name} to {device}')



Moved news_encoder to cuda
Moved bias_news_encoder to cuda
Moved pop_aware_user_encoder to cuda
Moved bias_scorer to cuda
Moved activity_gater to cuda
Moved time_embedding_layer to cuda
Moved time_distributed1 to cuda
Moved time_distributed2 to cuda
Moved time_distributed3 to cuda
Moved scaler to cuda
Moved softmax to cuda


In [167]:
colds = [0,1,3,5]
topKs = 10

In [168]:
from tqdm import tqdm
val_metrics_epoch = []
num_epochs = 10
# Step 2: Create your Adam optimizer
optimizer = Adam(model.parameters(), lr=0.0001)

loss_fn = nn.CrossEntropyLoss()

# Step 3: Iterate over the data for the number of epochs
for epoch in range(num_epochs):

# Step 4: Iterate over each batch of data and compute the scores using the forward pass of the network
    model.train()
    for x, y in tqdm(train_loader):
        optimizer.zero_grad()
        x= [i.to(device) for i in x]
        y = y.to(device)
        out = model(x)
    
        # Step 5: Compute the lambda gradient values for the pairwise loss (spedup) with the compute_lambda_i method on the scores and the output labels
        loss = loss_fn(out, y)

        # Step 6: Bacward from the scores with the use of the lambda gradient values
        if loss is not None:
            # torch.autograd.backward(out, loss)
            loss.backward()
            
            # Step 7: Update the weights using the optimizer
            optimizer.step()

    model.eval()

    val_metrics = eval_model(model_config, News, user_encoder, val_impressions, val_user_data, val_userids,
               news_encoder, bias_news_encoder, activity_gater, time_embedding_layer, 
               bias_content_scorer, scaler, colds, topKs, ValidUsers, device)
    
    print("epoch: {}, metrics: {}".format(epoch, val_metrics))
    val_metrics_epoch.append(val_metrics)

    if epoch > 1:
        if (val_metrics_epoch[-1][0] < val_metrics_epoch[-2][0]):
            break



100%|██████████| 2472/2472 [02:12<00:00, 18.59it/s]


epoch: 0, metrics: ([0.6871910644911863, 0.47177494445969925, 0.5272017179812674, 0.5778438550397963], [[0.658041865112312, 0.6666604127454792, 0.6353712696994431, 0.6758032825557174], [0.43360086320373364, 0.4642017977266868, 0.44934028758787, 0.4798196589182645], [0.4842713809603881, 0.5232790398414623, 0.4950079972475232, 0.5206428854250618], [0.5466489194715405, 0.5663811616162508, 0.5543808965603837, 0.5781086410670464]], [0.07369669005735524, 0.12336583607643478, 0.15260618946983817, 0.17944139726246208, 0.2030724011395488, 0.2166902677805819, 0.2295070834427306, 0.23511444029492068, 0.24152284812599503, 0.24552810302041653], [-5.5216276915713805e-09, 0.3024846142260388, 0.40107786407073337, 0.4518013589783394, 0.4798381378735539, 0.494579129451494, 0.5029805210030195, 0.508299704580824, 0.511329045187694, 0.5130973092150727], [0.9999999944783723, 0.6049692347091186, 0.5003327527791744, 0.456935083861297, 0.4240624223591633, 0.40464024596567294, 0.39253222373311186, 0.38455654140

  7%|▋         | 41/623 [00:32<07:37,  1.27it/s]


KeyboardInterrupt: 

In [ ]:
colds = [0,1,3,5]
topKs = 10
test_metrics, test_cold_metrics, test_topics, test_ILADs, test_ILMDs = eval_model(model_config, News, user_encoder, test_impressions, test_user_data, test_userids,
               news_encoder, bias_news_encoder, activity_gater, time_embedding_layer, 
               bias_content_scorer, scaler, colds, topKs, TestUsers, device)

100%|██████████| 2473/2473 [01:20<00:00, 30.72it/s]


In [ ]:
print("val metrics", val_metrics_epoch)
print("test metrics", test_metrics)
print("cold", test_cold_metrics)
print("diversity", [test_ILADs, test_ILMDs])

results = {"config": config,
           "model_config" : model_config,
            "val metrics": val_metrics_epoch, 
           "test metrics": test_metrics,
           "cold": test_cold_metrics,
           "diversity": [test_ILADs, test_ILMDs]}

import json
with open('results{}.json'.format(time.time()), 'w') as f:
    json.dump(results, f)

val metrics [([0.6825985432030286, 0.4678036431706821, 0.5215271382560431, 0.5732920996642947], [[0.6493983696935274, 0.6704979452350633, 0.6265554122658715, 0.714289747329046], [0.4341690672823048, 0.4483791889694272, 0.430428089405395, 0.5092703426698659], [0.4792058323738279, 0.5061599851492613, 0.4901250550959699, 0.5487523204934007], [0.5414497575635876, 0.5586277056188979, 0.5416699521297057, 0.6011163901916806]], [0.06208145086353296, 0.11415374981926535, 0.1485989419112901, 0.17503362421447186, 0.19986620455988505, 0.21669026778058184, 0.225101303058867, 0.23231076186882563, 0.23751759323157356, 0.24152284812599506], [-8.342721315649335e-09, 0.31706777133023467, 0.41907363067890446, 0.470505460180511, 0.49986247952219737, 0.5155224436020003, 0.5250930819889489, 0.5308018015474564, 0.5346818367640177, 0.5368398887054048], [0.9999999916572787, 0.6341355530044793, 0.5381401156867978, 0.5026904099943376, 0.4833037127231317, 0.4731364487685432, 0.4673265367264115, 0.4640735447358545

## Finished
## ---